In [1]:
import numpy as np
from tqdm import tqdm
import test_functions as test
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential, load_model

2023-05-06 15:36:26.157708: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 15:36:26.281158: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-06 15:36:26.283389: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-06 15:36:28.348143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
                        ################### PSO ######################

In [6]:
rosenbrock_model = load_model('../models/rosenbrock_model.hdf5')
hartman6D_model = load_model('../models/hartman6D_model.hdf5')
rastrigin_model = load_model('../models/rastrigin_model.hdf5')
ackley_model = load_model('../models/ackley_model.hdf5')

In [11]:
class Particle:
    global_best_value = None
    global_best_position = None
    
    def __init__(self, obj_func, bounds, c_i, c_s, c_c):
        self.obj_function = obj_func
        self.c_i = c_i
        self.c_c = c_c
        self.c_s = c_s
        
        self.lower_bounds = np.array([bound[0] for bound in bounds])
        self.upper_bounds = np.array([bound[1] for bound in bounds])
        
        position = np.random.uniform(self.lower_bounds, self.upper_bounds, len(bounds))
        position = position.reshape(1,-1)

        self.position = ackley_model.predict(position, verbose=False)
        self.value = self.obj_function(self.position[0])
        
        self.personal_best_position = self.position.copy()
        self.personal_best_value = self.value
        
        self.velocity = np.random.uniform(self.lower_bounds - self.upper_bounds,
                                          self.upper_bounds - self.lower_bounds,
                                          len(bounds))
        
        if Particle.global_best_value is None or self.value < Particle.global_best_value:
            Particle.global_best_position = self.position.copy()
            Particle.global_best_value = self.value
        
    
    def update_velocity(self):
        r_c = np.random.random(self.velocity.shape)
        r_s = np.random.random(self.velocity.shape)
        
        cognitive_velocity = r_c * self.c_c * (self.personal_best_position - self.position)
        social_velocity    = r_s * self.c_s * (Particle.global_best_position - self.position)
        inertia            = self.c_i * self.velocity
        
        self.velocity = cognitive_velocity + social_velocity + inertia
    
    def update_position(self):
        self.position = np.clip(self.position + self.velocity, self.lower_bounds, self.upper_bounds)
        self.value = self.obj_function(self.position[0])
        
        if self.value < self.personal_best_value:
            self.personal_best_value = self.value
            self.personal_best_position = self.position.copy()
            
            if self.value < Particle.global_best_value:
                Particle.global_best_value = self.value
                Particle.global_best_position = self.position.copy()

In [8]:
def PSO(swarm_size, num_iters, c_i, c_c, c_s, obj_func, num_dimensions, lb, ub):
    bounds = [(lb, ub) for _ in range(num_dimensions)]
    swarm = [Particle(obj_func, bounds, c_i, c_s, c_c) for _ in range(swarm_size)]
    
    for iter in tqdm(range(num_iters)):
        for particle in swarm:
            particle.update_velocity()
            particle.update_position()
        
    print(f'Global best value: {Particle.global_best_value}')
    print(f'Global best position: {Particle.global_best_position[0]}')
        
    Particle.global_best_position = None
    Particle.global_best_value = None

In [ ]:
                          ############# ROSENBROCK FUNCTION #############

In [84]:
# w/o using the model
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=0.85, c_s=1.0, obj_func=test.rosenbrock, num_dimensions=3, lb=-5, ub=10)

100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 388.93it/s]

Global best value: 6.472772238347563e-14
Global best position: [0.99999989 0.99999977 0.99999955]


In [80]:
# w/ using the model to predict good starting positions for particles
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=1.0, c_s=0.2, obj_func=test.rosenbrock, num_dimensions=3, lb=-5, ub=10)

100%|██████████████████████████████████████| 1000/1000 [00:02<00:00, 349.72it/s]

Global best value: 1.5210217065848887e-20
Global best position: [1. 1. 1.]


In [ ]:
                          ############# HARTMAN_6D FUNCTION #############

In [87]:
# w/o using the model
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=0.85, c_s=1.0, obj_func=test.hartman6D, num_dimensions=6, lb=0, ub=1)

100%|██████████████████████████████████████| 1000/1000 [00:06<00:00, 146.65it/s]

Global best value: -3.203102050264328
Global best position: [0.4046539  0.88245082 0.84650909 0.57399627 0.13837492 0.0384876 ]


In [99]:
# w/ using the model to predict good starting positions for particles
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=1.0, c_s=0.2, obj_func=test.hartman6D, num_dimensions=6, lb=0, ub=1)

100%|██████████████████████████████████████| 1000/1000 [00:06<00:00, 144.17it/s]

Global best value: -3.321994768543516
Global best position: [0.20179369 0.14675932 0.47663874 0.27542647 0.31167055 0.65725579]


In [ ]:
                          ############# RASTRIGIN FUNCTION #############

In [107]:
# w/o using the model
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=0.85, c_s=1.0, obj_func=test.rastrigin, num_dimensions=5, lb=-5.12, ub=5.12)

100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 316.86it/s]

Global best value: 5.969749304740667
Global best position: [-4.00442885e-10 -3.38652499e-09 -1.98991223e+00 -9.94958637e-01
  9.94958635e-01]


In [104]:
# w/ using the model to predict good starting positions for particles
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=1.0, c_s=0.2, obj_func=test.rastrigin, num_dimensions=5, lb=-5.12, ub=5.12)

100%|██████████████████████████████████████| 1000/1000 [00:04<00:00, 240.83it/s]

Global best value: 0.0002882320289359086
Global best position: [-0.00034654  0.0004227   0.00077964 -0.00071531 -0.00018594]


In [ ]:
                          ############# ACKLEY FUNCTION #############

In [5]:
# w/o using the model
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=0.85, c_s=1.0, obj_func=test.ackley, num_dimensions=9, lb=-32.768, ub=32.768)

100%|██████████████████████████████████████| 1000/1000 [00:03<00:00, 260.83it/s]

Global best value: 2.713178663222227
Global best position: [-9.55069088e-01 -9.55069099e-01 -9.55069201e-01  2.53432797e-08
  3.67363489e-09  9.55069099e-01  2.58769637e-09  9.55069097e-01
 -7.25223759e-09]


In [12]:
# w/ using the model to predict good starting positions for particles
PSO(swarm_size=50, num_iters=1000, c_i=0.4, c_c=1.0, c_s=0.2, obj_func=test.ackley, num_dimensions=9, lb=-32.768, ub=32.768)

100%|██████████████████████████████████████| 1000/1000 [00:04<00:00, 236.63it/s]

Global best value: 0.11332531767532261
Global best position: [-0.04616659 -0.01980186  0.01820415 -0.00454473 -0.01354281 -0.01444344
 -0.00140699 -0.00194729 -0.03270318]
